In [2]:
import pulp
import numpy as np

In [11]:
# Parámetros del problema
full_time_daily_hours = 8  # Horas diarias continuas para full-time
part_time_daily_hours = 4  # Horas diarias continuas para part-time
full_time_hours_per_week = 40  # Máximo de horas por semana para full-time
part_time_hours_per_week = 20  # Máximo de horas por semana para part-time
cost_full_time = 10 * full_time_hours_per_week  # Costo por semana para full-time
cost_part_time = 15 * part_time_hours_per_week  # Costo por semana para part-time

# Demanda horaria (matriz de 7 días x 13 horas)
demanda = np.array([
    [2, 2, 3, 3, 5, 5, 4],  # 8-10 AM
    [2, 2, 3, 3, 5, 5, 4],  # 10-12 PM
    [3, 3, 4, 4, 5, 6, 6],  # 12-14 PM
    [5, 5, 5, 5, 6, 8, 8],  # 14-16 PM
    [4, 4, 4, 4, 5, 6, 6],  # 16-18 PM
    [5, 5, 5, 5, 6, 7, 7],  # 18-21 PM
    [4, 4, 4, 4, 5, 6, 6]   # 21-23 PM
])

# Crear el problema de optimización
problem = pulp.LpProblem("Staffing_Problem", pulp.LpMinimize)

# Variables de decisión: número de contratos full-time y part-time por día y hora de inicio
full_time_shifts = pulp.LpVariable.dicts("FT_Shift", [(d, h) for d in range(7) for h in range(13 - full_time_daily_hours + 1)], 
                                         lowBound=0, cat='Integer')
part_time_shifts = pulp.LpVariable.dicts("PT_Shift", [(d, h) for d in range(7) for h in range(13 - part_time_daily_hours + 1)], 
                                         lowBound=0, cat='Integer')

# Función objetivo: minimizar el costo total
problem += pulp.lpSum([full_time_shifts[(d, h)] * cost_full_time for d in range(7) for h in range(13 - full_time_daily_hours + 1)]) + \
           pulp.lpSum([part_time_shifts[(d, h)] * cost_part_time for d in range(7) for h in range(13 - part_time_daily_hours + 1)]), "Total Cost"

# Restricciones: cubrir la demanda horaria diaria con los turnos
for d in range(7):  # Para cada día de la semana
    for h in range(7):  # Para cada hora (7 horas de la matriz demanda)
        # Cobertura por full-time: verificamos que la hora de inicio esté en el rango permitido
        ft_coverage = pulp.lpSum([full_time_shifts[(d, h_start)] for h_start in range(max(0, h - full_time_daily_hours + 1), min(h + 1, 13 - full_time_daily_hours + 1))])
        
        # Cobertura por part-time: verificamos que la hora de inicio esté en el rango permitido
        pt_coverage = pulp.lpSum([part_time_shifts[(d, h_start)] for h_start in range(max(0, h - part_time_daily_hours + 1), min(h + 1, 13 - part_time_daily_hours + 1))])
        
        # La cobertura total debe ser al menos la demanda en esa hora
        problem += ft_coverage + pt_coverage >= demanda[h, d], f"Coverage_day{d}_hour{h}"

# Resolver el problema
problem.solve()

1

In [12]:
# Imprimir resultados
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Coste mínimo: {pulp.value(problem.objective):.2f}")

# Imprimir los turnos asignados
print("Turnos Full-time:")
for d in range(7):
    for h in range(13 - full_time_daily_hours + 1):
        if pulp.value(full_time_shifts[(d, h)]) > 0:
            print(f"  Día {d}, inicio {h}: {pulp.value(full_time_shifts[(d, h)])} trabajador(es) full-time")

print("Turnos Part-time:")
for d in range(7):
    for h in range(13 - part_time_daily_hours + 1):
        if pulp.value(part_time_shifts[(d, h)]) > 0:
            print(f"  Día {d}, inicio {h}: {pulp.value(part_time_shifts[(d, h)])} trabajador(es) part-time")

Status: Optimal
Coste mínimo: 14900.00
Turnos Full-time:
  Día 0, inicio 0: 2.0 trabajador(es) full-time
  Día 1, inicio 0: 2.0 trabajador(es) full-time
  Día 2, inicio 0: 3.0 trabajador(es) full-time
  Día 3, inicio 0: 3.0 trabajador(es) full-time
  Día 4, inicio 0: 5.0 trabajador(es) full-time
  Día 5, inicio 0: 4.0 trabajador(es) full-time
  Día 6, inicio 0: 3.0 trabajador(es) full-time
  Día 6, inicio 1: 1.0 trabajador(es) full-time
Turnos Part-time:
  Día 0, inicio 2: 1.0 trabajador(es) part-time
  Día 0, inicio 3: 2.0 trabajador(es) part-time
  Día 1, inicio 2: 1.0 trabajador(es) part-time
  Día 1, inicio 3: 2.0 trabajador(es) part-time
  Día 2, inicio 2: 1.0 trabajador(es) part-time
  Día 2, inicio 3: 1.0 trabajador(es) part-time
  Día 3, inicio 2: 1.0 trabajador(es) part-time
  Día 3, inicio 3: 1.0 trabajador(es) part-time
  Día 4, inicio 2: 1.0 trabajador(es) part-time
  Día 5, inicio 0: 1.0 trabajador(es) part-time
  Día 5, inicio 2: 1.0 trabajador(es) part-time
  Día 5, inic